# WeatherPy - Itinerary Creation

### Dependencies and data

In [1]:
%matplotlib inline

import datetime as dt
import gmaps
import requests
import numpy as np
import pandas as pd

import config

In [2]:
# Configure API key
gmaps.configure(api_key=config.GMAPS_API_KEY)

In [3]:
# Hotel data
hotels = pd.read_csv('data/hotels.csv')
hotels.head(2)

,ID,City,Country,Latitude,Longitude,Time,Description,Max Temp,Humidity,Wind Speed,Cloudiness,Rain,Snow,Hotel
0,2,Butaritari,KI,3.0707,172.7902,2021-01-09 20:45:49,light rain,81.70,80,21.07,62,0.2200,0.0,Isles Sunset Lodge
1,26,San Policarpo,PH,12.1791,125.5072,2021-01-09 20:45:54,heavy intensity rain,80.11,85,19.33,100,5.0126,0.0,JM Lodge


### Narrow search and find 4-5 nearby cities

In [4]:
# Get the lowest and highest `Max Temp` in the data
min_mt, max_mt = hotels['Max Temp'].min(), hotels['Max Temp'].max()
min_mt, max_mt

(70.0, 89.6)

In [12]:
def take_input(prompt, min_temp=min_mt, max_temp=max_mt, yes_no=False):
    
    """
    Take user input for weather preferences.
    
    Args:
        [1] prompt (str) - input prompt
        [2] min_temp (float) - minimum temperature in data
        [3] max_temp (float) - maximum temperature in data
        [4] yes_no (bool) - whether the input should be "yes" or "no" only
        
    Returns:
        [float or str] User input
    """
    
    # Initialize user input and loop condition to False
    i = b = False
    
    if yes_no: # yes/no input
        while i not in ['yes', 'no']: # continue until input is "yes" or "no"
            i = input(prompt) # take user input
        i = 1 if i == 'yes' else 0 # convert input to numeric
    else: 
        while not b: # loop condition
            try:
                i = float(input(prompt)) # take user input and convert to numeric
            except:
                continue
            if min_temp <= i <= max_temp: # check if user input is within proper temperature range
                b = True # end loop
            
    # Numeric input
    return i
        
    
# Get preferred weather
min_temp = take_input('What is the lowest temperature you would like at your destination? ')
max_temp = take_input('What is the highest temperature you would like at your destination? ', min_temp=min_temp)
rain = take_input('Would you like it to rain at your destination? (yes/no) ', yes_no=True)
snow = take_input('Would you like it to snow at your destination? (yes/no) ', yes_no=True)
min_temp, max_temp, rain, snow # enter 75, 85, yes, no

What is the lowest temperature you would like at your destination? 75
What is the highest temperature you would like at your destination? 85
Would you like it to rain at your destination? (yes/no) yes
Would you like it to snow at your destination? (yes/no) no


(75.0, 85.0, 1, 0)

In [13]:
# Filter for max temp preference
pref_cities = hotels.loc[(hotels['Max Temp'] >= min_temp) & (hotels['Max Temp'] <= max_temp)]

# Filter for rain preference
if rain:
    pref_cities = pref_cities[pref_cities['Rain'] > 0]
else:
    pref_cities = pref_cities[pref_cities['Rain'] == 0]

# Filter for snow preference
if snow:
    pref_cities = pref_cities[pref_cities['Snow'] > 0]
else:
    pref_cities = pref_cities[pref_cities['Snow'] == 0]
    
pref_cities.shape

(46, 14)

In [14]:
# Request url and params
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
payload = {
    'key': config.GMAPS_API_KEY,
    'type': 'lodging', # hotels
    'radius': 5000 # meters
}

# Indices of cities without results
i404 = []

# Find nearest hotel for each city
for idx, row in pref_cities.iterrows():
    payload['location'] = f'{row["Latitude"]},{row["Longitude"]}'
    try:
        response = requests.get(base_url, params=payload)
        response_json = response.json()
        pref_cities.loc[idx, 'Hotel'] = response_json['results'][0]['name']
    except:
        i404.append(idx)

print(len(i404))
hotels.loc[i404, 'City'].tolist()

0


[]

### Temperature heatmap

In [15]:
# For Jupyterlab
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager
# !jupyter lab build

In [17]:
# Locations and measurements
locations = pref_cities[['Latitude', 'Longitude']]
weights = pref_cities['Max Temp'].apply(lambda t: max(t, 0))

# Info
hotel_info = '''
<dl>
<dt>Hotel</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Max Temp}F - {Description}</dd>
</dl>
'''

# Layers
heat_layer = gmaps.heatmap_layer(locations, weights, dissipating=False, point_radius=7, max_intensity=140)
mark_layer = gmaps.marker_layer(locations,
                                info_box_content=[hotel_info.format(**row) for idx, row in pref_cities.iterrows()])

# Heatmap
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=1.75)
fig.add_layer(heat_layer)
fig.add_layer(mark_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create travel itinerary

In [19]:
# Select cities
cities = ['Altamira', 'São Félix do Xingu', 'Miracema do Tocantins', 'São Miguel do Araguaia']
cities_df = pref_cities[pref_cities['City'].isin(cities)]
cities_df

,ID,City,Country,Latitude,Longitude,Time,Description,Max Temp,Humidity,Wind Speed,Cloudiness,Rain,Snow,Hotel
36,555,Miracema do Tocantins,BR,-9.5618,-48.3967,2021-01-09 20:47:55,light rain,79.27,78,2.89,100,0.200,0.0,Miami Apart Hotel
39,567,São Miguel do Araguaia,BR,-13.2750,-50.1628,2021-01-09 20:47:57,moderate rain,82.11,75,6.93,69,3.158,0.0,Hotel Executivo Palace
41,606,São Félix do Xingu,BR,-6.6447,-51.9950,2021-01-09 20:48:06,very heavy rain,83.37,71,3.56,100,23.679,0.0,Rio Xingu Camping
61,962,Altamira,BR,-3.2033,-52.2064,2021-01-09 20:49:26,moderate rain,78.84,92,2.10,99,2.526,0.0,Requinte Hotel


In [24]:
# Extract coordinates for each city
cities_df = cities_df.loc[[61, 39, 41, 36]]
locations = cities_df[['Latitude', 'Longitude']].values
locations

array([[ -3.2033, -52.2064],
       [-13.275 , -50.1628],
       [ -6.6447, -51.995 ],
       [ -9.5618, -48.3967]])

In [40]:
# Info
city_info = '''
<dl>
<dt>Hotel</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Max Temp} F - {Description}</dd>
</dl>
'''

# Layers
dir_layer = gmaps.directions_layer(locations[0], locations[1], locations[2:], show_markers=False)
mark_layer = gmaps.marker_layer(locations,
                                info_box_content=[city_info.format(**row) for idx, row in cities_df.iterrows()])

# Map
fig = gmaps.figure(center=(-6.0, -48.0), zoom_level=5)
fig.add_layer(dir_layer)
fig.add_layer(mark_layer)
fig

Figure(layout=FigureLayout(height='420px'))